Importing 4/2/18 data from District 4, along with its CHP incidents file, and metadata for those stations.

In [2]:
import pandas as pd
import numpy as np

In [2]:
five_min = pd.read_csv("./data/d04_text_station_5min_2018_04_02.txt.gz", compression="gzip", header=None)

In [3]:
chp_incidents = pd.read_csv("./data/all_text_chp_incident_day_2018_04_02.txt.gz", compression="gzip", header=None)
chp_detail = pd.read_csv("./data/all_text_chp_incident_det_day_2018_04_02.txt.gz", compression="gzip", header=None)

In [18]:
meta = pd.read_csv("./data/d04_text_meta_2018_04_13.txt", sep="\t")
meta.head()
# useful to connect ID # to number of lanes

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,400000,101,S,4,41,52582.0,19.23,459.835,38.081498,-122.547963,0.415,ML,4,N of 37 / 101 Interchange,DT5A2,L4-S-27-101-01923,101482.0,424150
1,400001,101,N,4,85,68000.0,38.26,387.897,37.364085,-121.901149,0.265,ML,5,NB 880 rm-n-loop,DT697,L5-N-37-101-03826,998145.0,426518
2,400002,101,S,4,81,68252.0,14.67,416.893,37.584097,-122.328465,0.310,ML,5,Peninsula Ave OC,DT243,L5-S-35-101-01467,106296.0,426945
3,400006,880,S,4,1,81204.0,13.54,23.767,37.605003,-122.065542,0.340,ML,4,Whipple Rd rm-s-diag,DTA20,L4-S-33-880-01354,998415.0,421858
4,400007,101,N,4,81,68252.0,15.25,417.437,37.586936,-122.337721,0.365,ML,5,2000' N of Pennisula Ave,DT275,L5-N-35-101-01525,999325.0,426427


In [14]:
colnames = five_min.columns
beginning_names = ["timestamp", "station", "district", "freeway", "direction", "length", "numSamples", "percentObs", "flow", "occupancy", "speed"]

new_names = beginning_names + list(colnames[11:])
five_min.columns = new_names

In [15]:
five_min.head()

,timestamp,station,district,freeway,direction,length,numSamples,percentObs,flow,occupancy,...,42,43,44,45,46,47,48,49,50,51
0,04/02/2018 00:00:00,400000,4,101,S,ML,0.415,40,100,0.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
1,04/02/2018 00:00:00,400001,4,101,N,ML,0.265,50,100,71.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2,04/02/2018 00:00:00,400002,4,101,S,ML,0.310,49,80,204.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
3,04/02/2018 00:00:00,400006,4,880,S,ML,0.340,40,75,127.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
4,04/02/2018 00:00:00,400007,4,101,N,ML,0.365,50,100,148.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0


---------

### Storing Data in SQL database: 

4-25-2018: I demonstrate how to write CSV files to a Sqlite3 database using [ClearingHouse data](http://pems.dot.ca.gov/?dnode=Clearinghouse) -- 5-minute link from 4/02/2018 on all districts. Download the file and store it in `./data` directory. Don't change the default name of the saved `.gz` file.

In [7]:
import csv, sqlite3

trafficDB = sqlite3.connect("traffic.sqlite")

#### IMPORTANT:

Only run this stuff once.

In [24]:
five_min = pd.read_csv("./data/contracted-Caltrans_text_gn_link_5min_2018_04_02.txt.gz", compression="gzip", header=None)

# always clean tables before writing into the database!
# remove some columns:
five_min = five_min.iloc[:,0:12]
#  manually supply column names:
five_min.columns = ["utc_time_id", "source_ref", "source_id","feed_id","primary_link_source_flag", "samples", "avg_speed", "avg_flow", "avg_occ", "avg_freeflow_speed" , "avg_travel_time", "high_quality_samples"] 

In [26]:
# write to DB
five_min.to_sql("Clear_4_02_2018", trafficDB, if_exists='replace')

In [34]:
# Check if the table was successfully added
# (printing all tables in the DB)
res = trafficDB.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

Clear_4_02_2018
